In [1]:
from keras_segmentation.models.fcn import fcn_8, fcn_32, fcn_8_vgg, fcn_32_vgg, fcn_8_resnet50, fcn_32_resnet50

ModuleNotFoundError: No module named 'keras_segmentation'

In [32]:
import matplotlib.pyplot as plt
from shutil import copyfile
import cv2
import os
import re
from google.colab import files
import tensorflow as tf
from zipfile import ZipFile
import os
import shutil
from os.path import basename
from timeit import default_timer as timer

In [31]:
# This is required for me to have autocomplete
%config Completer.use_jedi = False
%matplotlib inline

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
## test GPU enabled
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [33]:
gdrive = False

In [34]:
# paths for normal use
if gdrive == False:
    base_path = "../"
else:
    base_path = "/content/gdrive/MyDrive/data_science/MBA/Thesis/"

In [35]:
# paths
    
# set path to trainset
data_path = base_path+"data/trainset/"

# set path for checkpoints
checkpoint_path = base_path+"data/train_checkpoints/"

# set path for final checkpoints
trainset_final_path = base_path+"data/train_final_checkpoints/"

# dataset sizes (for looping)
sizes = ["small","medium","large","small_transform","medium_transform","large_transform"]

In [37]:
# set model parameters
model_params = {"input_height": 800,
                "input_width": 800,
                "n_classes" : 2}

In [38]:
# set models
Vanilla8 = fcn_8(**model_params)
Vanilla32 = fcn_32(**model_params)
VGG8 = fcn_8_vgg(**model_params)
VGG32 = fcn_32_vgg(**model_params)
Resnet8 = fcn_8_resnet50(**model_params)
Resnet32 = fcn_32_resnet50(**model_params)

In [39]:
# create model dict
models = {"Vanilla8": Vanilla8,
         "Vanilla32": Vanilla32,
         "VGG8": VGG8,
         "VGG32": VGG32,
         "Restnet8": Resnet8,
         "Restnet32": Resnet32}

In [53]:
# make directories for training
for model in models.keys():
    for size in sizes:
        size = "/"+size
        os.makedirs(checkpoint_path+model+size, exist_ok=True)

In [54]:
# create folder tree for final epoch predictions
for model in models.keys():
    path = trainset_final_path+model
    os.makedirs(path, exist_ok=True)

In [80]:
# train models
for model_name, model in models.items():
    for size in sizes:
        print("training model "+model_name+" with dataset size "+size)
        train_path = data_path+size+"/images/"
        annotation_path = data_path+size+"/annotations/"
        checkpnt_path = checkpoint_path+model_name+"/"+size+"/"+model_name+"_"+size+"_weights"
        
        model.train(
            train_images =  train_path,
            train_annotations = annotation_path,
            checkpoints_path = checkpnt_path , 
            epochs=5,
            verify_dataset=False
        )
        shutil.make_archive(base_path+"data/train_checkpoints/"+model_name, 'zip',
                    base_path+"data/train_checkpoints/+model_name)

training model Vanilla8 with dataset size large_transform
Epoch 1/5
512/512 [==============================] - 1867s 4s/step - loss: 0.0053 - accuracy: 0.9985

Epoch 00001: saving model to ../data/train_checkpoints/Vanilla8/large_transform/Vanilla8_large_transform_weights.00001
Epoch 2/5
512/512 [==============================] - 1871s 4s/step - loss: 0.0038 - accuracy: 0.9987

Epoch 00002: saving model to ../data/train_checkpoints/Vanilla8/large_transform/Vanilla8_large_transform_weights.00002
Epoch 3/5
512/512 [==============================] - 1837s 4s/step - loss: 0.0040 - accuracy: 0.9986

Epoch 00003: saving model to ../data/train_checkpoints/Vanilla8/large_transform/Vanilla8_large_transform_weights.00003
Epoch 4/5
512/512 [==============================] - 1837s 4s/step - loss: 0.0031 - accuracy: 0.9989

Epoch 00004: saving model to ../data/train_checkpoints/Vanilla8/large_transform/Vanilla8_large_transform_weights.00004
Epoch 5/5
512/512 [==============================] - 1834s

KeyboardInterrupt: 

In [29]:
# loop over training files and copy final epoch
for model in ["VGG32", "VGG8", "Vanilla8", "Vanilla32"]:
    for size in sizes:
        
        in_files = checkpoint_path+model+"/"+size+"/"
        out_files = trainset_final_path+model+"/"+size+"/"
        
        files = os.listdir(in_files)
        f_file = [f for f in files if "00005." in f]
        
        for f in f_file:
            shutil.copy(in_files+f, out_files+f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_checkpoints/Vanilla8/small/'

In [75]:
for model_name in ["Vanilla8"]:#["VGG32", "VGG8", "Vanilla32", "Restnet8","Restnet32"]:
    zipfile_path = checkpoint_path+model_name+".zip"

    with ZipFile(zipfile_path, 'r') as zipObj:
        listOfFileNames = zipObj.namelist()
        list1 = [f for f in listOfFileNames if "00005." in f]
        list2 = [f for f in listOfFileNames if ".json" in f]
        finallist = list1+list2
        print(finallist)
        
        # Restnet 8 and 32 is zipped differently
        for f in finallist:
            if model_name in ["Restnet8","Restnet32"]:
                zipObj.extract(f ,'../data/train_final_checkpoints2/')
            else:
                zipObj.extract(f ,'../data/train_final_checkpoints2/'+model_name)

['small/Vanilla8_small_weights.00005.data-00000-of-00001', 'small/Vanilla8_small_weights.00005.index', 'medium/Vanilla8_medium_weights.00005.data-00000-of-00001', 'medium/Vanilla8_medium_weights.00005.index', 'large/Vanilla8_large_weights.00005.data-00000-of-00001', 'large/Vanilla8_large_weights.00005.index', 'small_transform/Vanilla8_small_transform_weights.00005.data-00000-of-00001', 'small_transform/Vanilla8_small_transform_weights.00005.index', 'medium_transform/Vanilla8_medium_transform_weights.00005.data-00000-of-00001', 'medium_transform/Vanilla8_medium_transform_weights.00005.index', 'large_transform/Vanilla8_large_transform_weights.00005.data-00000-of-00001', 'large_transform/Vanilla8_large_transform_weights.00005.index', 'small/Vanilla8_small_weights_config.json', 'medium/Vanilla8_medium_weights_config.json', 'large/Vanilla8_large_weights_config.json', 'small_transform/Vanilla8_small_transform_weights_config.json', 'medium_transform/Vanilla8_medium_transform_weights_config.js

In [25]:
# loop over prediction files and copy final epoch
for model in ["VGG32", "VGG8", "Vanilla8", "Vanilla32"]:
    for size in sizes:
        
        zipfile_path = checkpoint_path+model
        
        with ZipFile(zipfile_path, 'r') as zipObj:
        
        in_files = checkpoint_path+model+"/"+size+"/"
        out_files = trainset_final_path+model+"/"+size+"/"
        
        files = os.listdir(in_files)
        f_file = [f for f in files if f.endswith("config.json")]
        
        for f in f_file:
            shutil.copy(in_files+f, out_files+f)